In [ ]:
import sys
import argparse
import requests
import os
from PIL import Image, ImageFilter

google drive 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Kakao 계정 연결 

In [ ]:
API_URL = 'https://dapi.kakao.com/v2/vision/face/detect'
MYAPP_KEY = '89912886a29c936fb657d80f648ac117' #본인 키 발급받은 거 넣기

face detection

In [ ]:
def detect_face(filename):
  headers = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}

  try:
      files = { 'image' : open(filename, 'rb')}
      resp = requests.post(API_URL, headers=headers, files=files)
      resp.raise_for_status()
      return resp.json()
  except Exception as e:
      print(str(e))
      sys.exit(0)

face crop

In [ ]:
def crop(filename, detection_result):
    image = Image.open(filename)
    IMAGE_SIZE = 100,100

    for face in detection_result['result']['faces']:
        x = int(face['x']*image.width)
        w = int(face['w']*image.width)
        y = int(face['y']*image.height)
        h = int(face['h']*image.height)
        box = image.crop((x,y,x+w, y+h))
        # box = box.resize((20,20), Image.NEAREST).resize((w,h), Image.NEAREST)
        # image.paste(box, (x,y,x+w, y+h))
        image = box.resize(IMAGE_SIZE,Image.ANTIALIAS)
    return image

이미지 파일에 적용

In [ ]:
# 결과 저장 폴더 생성
out_dir ="crop_profile"
if out_dir not in os.listdir():
    os.mkdir(out_dir)

if __name__ == "__main__":
    img_dir = '/content/drive/MyDrive/ghibliimagedataset/ghibli_profileimage'
    imagefile = os.listdir(os.path.join(img_dir))
    os.listdir(img_dir)

    for i in imagefile:
      try:
        detection_result = detect_face(img_dir +'/'+ i)
        image = crop(img_dir +'/'+ i, detection_result)  
        image.save(out_dir + "/" + i)  
        print('save%s' %i)
      except:
        pass